In [2]:
from pywebio.input import *
from pywebio.output import *

import pandas as pd
import numpy as np

In [3]:
df_cleaned = pd.read_csv('cleaned_v2.csv')
df = pd.read_csv('all_data.csv')

In [4]:
types=np.unique(df_cleaned['type'])

In [5]:
df_cleaned.head()

,Unnamed: 0,price,rating,type,title,dine_in,takeout,delivery,price_num
0,2,$,4.2,turkish,Divan Turkish Restaurant,1.0,1.0,1.0,1
1,7,$$,4.0,italian,"Restaurante La Tagliatella | Diagonal Mar, Bar...",1.0,1.0,1.0,2
2,11,$,4.6,vegan_vegetarian,La Trocadero,1.0,1.0,1.0,1
3,12,$$$,4.4,spanish,El Nacional,1.0,1.0,1.0,3
4,15,$,4.3,asian,XiAn Restaurante,1.0,1.0,1.0,1


In [6]:
#location = '{41.38386,2.11663}'

In [7]:
df_dist = pd.read_csv('AAAAAIIIII.csv')

In [7]:
df_dist.head()

,Unnamed: 0,Nombre,Distancia,Posicion
0,0,Restaurant Tritón,0.552306,"[41.3867537, 2.1125587]"
1,1,Santa Clara Restaurant,0.830799,"[41.389526, 2.1215645]"
2,2,De Angelis,0.931240,"[41.3758353, 2.1191858]"
3,3,Futballárium Barcelona,1.176793,"[41.3777839, 2.125367599999999]"
4,4,Punta Anguila,1.639249,"[41.3821582, 2.1313569]"


In [8]:
names=df_dist['Nombre']

In [9]:
df_dist[df_dist['Nombre']==df_cleaned['title'][0]]['Distancia']

132    5.917259
133    5.917259
134    5.917259
Name: Distancia, dtype: float64

In [10]:
dist = []
for i in range(len(df_cleaned)):
    dist.append((df_dist[df_dist['Nombre']==df_cleaned['title'][i]]['Distancia'].values))
    #if df_cleaned['title'][i] in names:
        #dist.append(df_dist[df_dist['Nombre']==df_cleaned['title'][i]]['Distancia'])

In [11]:
dist[0][0]

5.917259409102655

In [12]:
len(dist)

89

In [13]:
d_n=[]
for d in dist:
    if len(d)>0:
        d_n.append(d[0])
    else:
        d_n.append(float("NaN"))

In [14]:
df_cleaned['dist']=d_n

In [15]:
df_cleaned.head()

,Unnamed: 0,price,rating,type,title,dine_in,takeout,delivery,price_num,dist
0,2,$,4.2,turkish,Divan Turkish Restaurant,1.0,1.0,1.0,1,5.917259
1,7,$$,4.0,italian,"Restaurante La Tagliatella | Diagonal Mar, Bar...",1.0,1.0,1.0,2,11.376802
2,11,$,4.6,vegan_vegetarian,La Trocadero,1.0,1.0,1.0,1,6.713869
3,12,$$$,4.4,spanish,El Nacional,1.0,1.0,1.0,3,5.771540
4,15,$,4.3,asian,XiAn Restaurante,1.0,1.0,1.0,1,6.796732


In [16]:
len(df_cleaned)

89

In [17]:
sent= df['description'].to_list()

In [18]:
from gensim.models import Word2Vec
from gensim.test.utils import lee_corpus_list

from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

data = []
  
# iterate through each sentence in the file
for s in sent:
    if type(s)==type('s'):
        for i in sent_tokenize(s):
            temp = []
      
    # tokenize the sentence into words
        for j in word_tokenize(i):
            if len(j)>1:
                temp.append(j.lower())
  
        data.append(temp)
data.append(lee_corpus_list[0])    
    

model = Word2Vec(sentences=data, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [19]:
sent=[]
for d in data:
    for i in d:
        sent.append(i)

In [20]:
#let people input words and get similarity
input("What's your name?")
serv = select("Choose a service option", ['dine_in','takeout','delivery'])
pos_res = df_cleaned[df_cleaned[serv]==1]
words = input("Tell me something about the restaurant you'd like to eat at!")
sim = []
for d in sent:
    for w in words:
        if w ==d:
            sim.append(model.wv.similarity(w, d))
        else:
            sim.append(0)
sim=sum(sim)
hunger_lvl = radio("How hungry are you?", options=['A little', 'Average', 'Very'])
coef_h = 1
if hunger_lvl=='A little':
    coef_h=0.8
elif hunger_lvl=='Very':
    coef_h=1.5
    
#type = select("Select the food type", types)
typ = checkbox("Select the food type(s) you'd like", options=types)
pos_res = df_cleaned[df_cleaned['type'].isin(typ)]
min_rat = radio("Choose a minimum rating", options=['1', '2', '3','4','5'])
max_price = radio("From 1 to 4, 4 being the most expensive, choose the maximum price", options=['1', '2', '3','4'])
pos_res = pos_res[pos_res['price_num']<=int(max_price)]
min_rat = int(min_rat)
pos_res = pos_res[pos_res['rating']>=min_rat]
rank = [-d*coef_h+sim for d in pos_res['dist']]
rank = rank+pos_res['rating']
pos_res['rank']=rank
pos_res=pos_res.sort_values(by=['rank'],ascending=False)
select("Here are the restaurants ranked by affinity, choose one!", pos_res['title'])
#res = select("Select restaurant", pos_res['title'])


'Five Guys'

3.0

In [61]:
sent= df['description'].to_list()

In [84]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

data = []
  
# iterate through each sentence in the file
for s in sent:
    if type(s)==type('s'):
        for i in sent_tokenize(s):
            temp = []
      
    # tokenize the sentence into words
        for j in word_tokenize(i):
            if len(j)>1:
                temp.append(j.lower())
  
        data.append(temp)

In [85]:
from gensim.test.utils import lee_corpus_list

In [86]:
data.append(lee_corpus_list[0])

In [21]:
pos_res

,Unnamed: 0,price,rating,type,title,dine_in,takeout,delivery,price_num,dist,rank
20,64,$$,4.3,grill,Restaurant ASADOR DE ARANDA,1.0,1.0,1.0,2,3.954100,0.345900
24,83,$$,3.9,american,Foster's Hollywood,1.0,1.0,1.0,2,4.995244,-1.095244
25,88,$$,4.3,hamburger,Five Guys,1.0,1.0,1.0,2,5.842565,-1.542565
79,210,$$,4.2,hamburger,Five Guys,1.0,1.0,1.0,2,5.842565,-1.642565
44,131,$$,4.1,hamburger,Five Guys,1.0,1.0,1.0,2,5.842565,-1.742565
88,238,$$,4.1,hamburger,Five Guys,1.0,1.0,1.0,2,5.842565,-1.742565
82,217,$,4.0,fast food,Restaurant KFC,1.0,1.0,1.0,1,8.179334,-4.179334


In [8]:
df_prueba = pd.read_csv('df_prueba.csv')

In [28]:
df_prueba

,Unnamed: 0,Nombre,Distancia,Posicion,tipo
0,0,Restaurant Tritón,0.552306,"(2.1125587, 41.3867537)",restaurant
1,1,Santa Clara Restaurant,0.830799,"(2.1215645, 41.389526)",bar & grill
2,2,De Angelis,0.931240,"(2.1191858, 41.3758353)",italian
3,3,Futballárium Barcelona,1.176793,"(2.125367599999999, 41.3777839)",restaurant
4,4,Punta Anguila,1.639249,"(2.1313569, 41.3821582)",restaurant
...,...,...,...,...,...
301,301,"Restaurante La Tagliatella | Diagonal Mar, Bar...",11.376802,"(2.2164796, 41.408681699999995)",italian
302,302,"Restaurante La Tagliatella | Diagonal Mar, Bar...",11.376802,"(2.2164796, 41.408681699999995)",italian
303,303,Restaurant Pacífic,11.654766,"(2.2177987, 41.413433)",restaurant
304,304,Bar Restaurant Nou,11.659553,"(2.2183364, 41.4116917)",restaurant
